# Mounting Azure storage account containers in Databricks

> Databricks recommends not to use mount points in a production setup for security reasons. These setup instructions here are provided as a quick solution for quickstart environments.

## Steps to complete before running the notebook
1. Search for "Microsoft Entra ID" in the Azure portal and select the corresponding service.
2. Select "Manage > App registrations" and list "All applications".
3. Add a new app registration, Account type "Single tenant", e.g. named "DatabricksEntraIdApp".
4. Select "Manage > Certificates & secrets".
5. Add new client secret to the app.
6. Copy and temporarily store the secret value. It will be disabled only right after the creation of the client secret.
7. Get "Application (client) ID" and "Directory (tenant) Id" of the app. See Overview.
8. Create an Azure Key Vault.
9. Add role "Key Vault Administrator" to your user.
10. Add secrets, see keys in the notebook.
11. Add Role "Key Vault Administrator" to app "AzureDatabricks".
12. Add Secret Scope in Databricks via https://YOUR-DATABRICKS-HOST/#secrets/createScope, get DNS Name (Vault URI) and Resource ID from the Azure Key Vault properties.
13. Add the created Microsoft Entra ID app as "Storage Blob Data Contributor" to the container(s).
14. Change the name of the storage account and the list of the containers in the notebook.

## Links to additional information
- https://learn.microsoft.com/en-us/azure/databricks/getting-started/connect-to-azure-storage
- https://learn.microsoft.com/en-us/answers/questions/1290398/not-able-to-create-secrete-in-azure-key-valut

In [ ]:
# Set the name of the storage account and a list of containers to be mounted.
adls_account_name = "STORAGE_ACCOUNT"
adls_containers = ["CONTAINER-1", "CONTAINER-2"]

application_id = dbutils.secrets.get(scope="kv-dbw-secrets", key="sp-client-id")          # Application (client) ID
authentication_key = dbutils.secrets.get(scope="kv-dbw-secrets", key="sp-client-secret")  # Client secret value
tenant_id = dbutils.secrets.get(scope="kv-dbw-secrets", key="sp-tenant-id")               # Directory (tenant) ID

configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": application_id,
           "fs.azure.account.oauth2.client.secret": authentication_key,
           "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}

for adls_container_name in adls_containers:
  mount_point = f"/mnt/{adls_account_name}/{adls_container_name}"
  source = f"abfss://{adls_container_name}@{adls_account_name}.dfs.core.windows.net/"
  
  # Mount only if the directory is not already mounted
  if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
    dbutils.fs.mount(
      source = source,
      mount_point = mount_point,
      extra_configs = configs)

In [ ]:
dbutils.notebook.exit('Azure datalake storage containers mounted.')

In [ ]:
for adls_container_name in adls_containers:
  dbutils.fs.ls(f"/mnt/{adls_account_name}/{adls_container_name}")

In [ ]:
dbutils.fs.mounts()